# Using `PacMan`

This notebook contains examples on how to use the class `PacMan`.

In [1]:
from pytket import Circuit
from pytket.circuit import OpType, Op
from pytket.circuit.display import render_circuit_jupyter
from pytket_dqc.networks import NISQNetwork
from pytket_dqc.allocators import HypergraphPartitioning
from pytket_dqc.circuits import HypergraphCircuit
from pytket_dqc.packing import PacMan
from pytket_dqc.utils import DQCPass
from pytket_dqc.placement import Placement
from sympy import Symbol
import networkx as nx
import matplotlib.pyplot as plt

cz = Op.create(OpType.CU1, 1)

circuit = Circuit(6)
mapping = list(range(6))
placement_dict = dict()
for i in range(6):
    placement_dict[i] = i // 2

`PacMan` can find all sorts of `CU1` gates that can be distributed together.  
For the purposes of these examples, we imagine that the circuit is distributed across 3 servers, each with 2 qubits.  
The qubits 0 & 1 are on server 0, qubits 2 & 3 on server 1, qubits 4 & 5 on server 2.  
The `Placements` are manually constructed to force this desired behaviour.

The simplest examples are `CU1` gates separated by single qubit diagonal gates.

In [2]:
# Custom placement
placement_dict00 = dict()
for i in range(6):
    placement_dict00[i] = i // 2
placement_dict00[6] = 0
placement_dict00[7] = 0
placement_dict[6] = 0
placement_dict[7] = 0
placement00 = Placement(placement_dict00)

# Build circuit
circ00 = Circuit(6)
circ00.add_gate(cz, [0,2])
circ00.Rz(0.3, 0)
circ00.add_gate(cz, [0,3])
circuit.add_circuit(circ00, mapping)
render_circuit_jupyter(circ00)

# Make hypergraph of circuit
h_circ00 = HypergraphCircuit(circ00)

# Pass into PacMan
pacman00 = PacMan(h_circ00, placement00)

print(pacman00.neighbouring_packets)

{0: [(P0, P1)], 1: [], 2: [], 3: [], 4: [], 5: []}


`CU1` gates separated by single qubit anti-diagonal gates can also be distributed together.

In [3]:
# Custom placement
placement_dict01 = dict()
for i in range(6):
    placement_dict01[i] = i // 2
placement_dict01[6] = 0
placement_dict01[7] = 0
placement_dict[8] = 0
placement_dict[9] = 0
placement01 = Placement(placement_dict01)

# Build circuit
circ01 = Circuit(6)
circ01.add_gate(cz, [0,2])
circ01.Rz(0.3, 0)
# Note H Rz(1) H = H Z H = X, an anti-diagonal gate.
circ01.H(0)
circ01.Rz(1, 0)
circ01.H(0)
circ01.add_gate(cz, [0,3])
circuit.add_circuit(circ01, mapping)
render_circuit_jupyter(circ01)

# Make hypergraph of circuit
h_circ01 = HypergraphCircuit(circ01)

# Pass into PacMan
pacman01 = PacMan(h_circ01, placement01)

print(pacman01.neighbouring_packets)

{0: [(P0, P1)], 1: [], 2: [], 3: [], 4: [], 5: []}


`CU1` gates that go to different servers are split into different packets.

In [4]:
# Custom placement
placement_dict02 = dict()
for i in range(6):
    placement_dict02[i] = i // 2
placement_dict02[6] = 0
placement_dict02[7] = 0
placement_dict02[8] = 0
placement_dict02[9] = 0
placement_dict[10] = 0
placement_dict[11] = 0
placement_dict[12] = 0
placement_dict[13] = 0
placement02 = Placement(placement_dict02)

# Build circuit
circ02 = Circuit(6)
circ02.add_gate(cz, [0,2])
circ02.add_gate(cz, [0,4])
circ02.Rz(0.5, 0)
circ02.add_gate(cz, [0,3])
circ02.add_gate(cz, [0,5])
circuit.add_circuit(circ02, mapping)
render_circuit_jupyter(circ02)

# Make hypergraph of circuit
h_circ02 = HypergraphCircuit(circ02)

# Pass into PacMan
pacman02 = PacMan(h_circ02, placement02)

print(pacman02.neighbouring_packets)

{0: [(P0, P2), (P1, P3)], 1: [], 2: [], 3: [], 4: [], 5: []}


The above 3 examples demonstrate neighbouring packing.  
Provided the right conditions are met, we can also merge packets via hopping packing.

The simplest case of this is two Hadamards surrounding a CZ gate (= CX gate).

In [5]:
# Custom placement
placement_dict03 = dict()
for i in range(6):
    placement_dict03[i] = i // 2
placement_dict03[6] = 0
placement_dict03[7] = 0
placement_dict03[8] = 0
placement_dict[14] = 0
placement_dict[15] = 0
placement_dict[16] = 0
placement03 = Placement(placement_dict03)

# Build circuit
circ03 = Circuit(6)
circ03.add_gate(cz, [0,2])
circ03.H(0)
circ03.add_gate(cz, [0,3])
circ03.H(0)
circ03.add_gate(cz, [0,2])
circuit.add_circuit(circ03, mapping)
render_circuit_jupyter(circ03)

# Make hypergraph of circuit
h_circ03 = HypergraphCircuit(circ03)

# Pass into PacMan
pacman03 = PacMan(h_circ03, placement03)

print(pacman03.hopping_packets)

{0: [(P0, P2)], 1: [], 2: [], 3: [], 4: [], 5: []}


More complicated hopping packings can also be identified.

Two embedded `CU1`s with a single `Z` between them.

In [6]:
# Custom placement
placement_dict04 = dict()
for i in range(6):
    placement_dict04[i] = i // 2
placement_dict04[6] = 0
placement_dict04[7] = 0
placement_dict04[8] = 0
placement_dict04[9] = 0
placement_dict[17] = 0
placement_dict[18] = 0
placement_dict[19] = 0
placement_dict[20] = 0
placement04 = Placement(placement_dict04)

# Build circuit
circ04 = Circuit(6)
circ04.add_gate(cz, [0,2])
circ04.H(0)
circ04.add_gate(cz, [0,2])
circ04.Rz(1, 0)
circ04.add_gate(cz, [0,3])
circ04.H(0)
circ04.add_gate(cz, [0,2])
circuit.add_circuit(circ04, mapping)
render_circuit_jupyter(circ04)

# Make hypergraph of circuit
h_circ04 = HypergraphCircuit(circ04)

# Pass into PacMan
pacman04 = PacMan(h_circ04, placement04)

print(pacman04.hopping_packets)

{0: [(P0, P3)], 1: [], 2: [], 3: [], 4: [], 5: []}


Or with an `X` between them.

In [7]:
# Custom placement
placement_dict05 = dict()
for i in range(6):
    placement_dict05[i] = i // 2
placement_dict05[6] = 0
placement_dict05[7] = 0
placement_dict05[8] = 0
placement_dict05[9] = 0
placement_dict[21] = 0
placement_dict[22] = 0
placement_dict[23] = 0
placement_dict[24] = 0
placement05 = Placement(placement_dict05)

# Build circuit
circ05 = Circuit(6)
circ05.add_gate(cz, [0,2])
circ05.H(0)
circ05.add_gate(cz, [0,2])
# Note H Rz(1) H = H Z H = X, an anti-diagonal gate.
circ05.H(0)
circ05.Rz(1, 0)
circ05.H(0)
circ05.add_gate(cz, [0,3])
circ05.H(0)
circ05.add_gate(cz, [0,2])
circuit.add_circuit(circ05, mapping)
render_circuit_jupyter(circ05)

# Make hypergraph of circuit
h_circ05 = HypergraphCircuit(circ05)

# Pass into PacMan
pacman05 = PacMan(h_circ05, placement05)

print(pacman05.hopping_packets)

{0: [(P0, P3)], 1: [], 2: [], 3: [], 4: [], 5: []}


(Actually in this case the `Rz` can have any phase)

In [8]:
# Custom placement
placement_dict06 = dict()
for i in range(6):
    placement_dict06[i] = i // 2
placement_dict06[6] = 0
placement_dict06[7] = 0
placement_dict06[8] = 0
placement_dict06[9] = 0
placement_dict[25] = 0
placement_dict[26] = 0
placement_dict[27] = 0
placement_dict[28] = 0
placement06 = Placement(placement_dict06)

# Build circuit
circ06 = Circuit(6)
circ06.add_gate(cz, [0,2])
circ06.H(0)
circ06.add_gate(cz, [0,2])
circ06.H(0)
circ06.Rz(0.27, 0)
circ06.H(0)
circ06.add_gate(cz, [0,3])
circ06.H(0)
circ06.add_gate(cz, [0,2])
circuit.add_circuit(circ06, mapping)
render_circuit_jupyter(circ06)

# Make hypergraph of circuit
h_circ06 = HypergraphCircuit(circ06)

# Pass into PacMan
pacman06 = PacMan(h_circ06, placement06)

print(pacman06.hopping_packets)

{0: [(P0, P3)], 1: [], 2: [], 3: [], 4: [], 5: []}


There may just be 1 Hadamard between two embedded `CU1`s.

In [9]:
# Custom placement
placement_dict07 = dict()
for i in range(6):
    placement_dict07[i] = i // 2
placement_dict07[6] = 0
placement_dict07[7] = 0
placement_dict07[8] = 0
placement_dict07[9] = 0
placement_dict[29] = 0
placement_dict[30] = 0
placement_dict[31] = 0
placement_dict[32] = 0
placement07 = Placement(placement_dict07)

# Build circuit
circ07 = Circuit(6)
circ07.add_gate(cz, [0,2])
circ07.H(0)
circ07.add_gate(cz, [0,2])
circ07.Rz(0.5, 0)
circ07.H(0)
circ07.Rz(0.5, 0)
circ07.add_gate(cz, [0,3])
circ07.H(0)
circ07.add_gate(cz, [0,2])
circuit.add_circuit(circ07, mapping)
render_circuit_jupyter(circ07)

# Make hypergraph of circuit
h_circ07 = HypergraphCircuit(circ07)

# Pass into PacMan
pacman07 = PacMan(h_circ07, placement07)

print(pacman07.hopping_packets)

{0: [(P0, P3)], 1: [], 2: [], 3: [], 4: [], 5: []}


In total we can have up to 5 single qubit gates between two CRz gates (this is guranteed by `DQCPass`!), and providing they satisfy the right conditions `PacMan` will find (most of) the embeddings!

In [10]:
# Custom placement
placement_dict08 = dict()
for i in range(6):
    placement_dict08[i] = i // 2
placement_dict08[6] = 0
placement_dict08[7] = 0
placement_dict08[8] = 0
placement_dict08[9] = 0
placement_dict[33] = 0
placement_dict[34] = 0
placement_dict[35] = 0
placement_dict[36] = 0
placement08 = Placement(placement_dict08)

# Build circuit
circ08 = Circuit(6)
circ08.add_gate(cz, [0,2])
circ08.H(0)
circ08.Rz(0.33, 0)
circ08.add_gate(cz, [0,2])
circ08.Rz(0.67, 0)
circ08.H(0)
circ08.Rz(0.27, 0)
circ08.H(0)
circ08.Rz(0.43847, 0)
circ08.add_gate(cz, [0,3])
circ08.Rz(1 - 0.43847, 0)
circ08.H(0)
circ08.add_gate(cz, [0,2])
circuit.add_circuit(circ08, mapping)
render_circuit_jupyter(circ08)

# Make hypergraph of circuit
h_circ08 = HypergraphCircuit(circ08)

# Pass into PacMan
pacman08 = PacMan(h_circ08, placement08)

print(pacman08.hopping_packets)

{0: [(P0, P3)], 1: [], 2: [], 3: [], 4: [], 5: []}


Embedding broken by third server gate.

In [11]:
# Custom placement
placement_dict09 = dict()
for i in range(6):
    placement_dict09[i] = i // 2
placement_dict09[6] = 0
placement_dict09[7] = 0
placement_dict09[8] = 0
placement_dict09[9] = 0
placement_dict[37] = 0
placement_dict[38] = 0
placement_dict[39] = 0
placement_dict[40] = 0
placement09 = Placement(placement_dict09)

# Build circuit
circ09 = Circuit(6)
circ09.add_gate(cz, [0,2])
circ09.H(0)
circ09.add_gate(cz, [0,2])
circ09.Rz(1, 0)
circ09.add_gate(cz, [0,4])
circ09.H(0)
circ09.add_gate(cz, [0,2])
circuit.add_circuit(circ09, mapping)
render_circuit_jupyter(circ09)

# Make hypergraph of circuit
h_circ09 = HypergraphCircuit(circ09)

# Pass into PacMan
pacman09 = PacMan(h_circ09, placement09)

print(pacman09.hopping_packets)

{0: [], 1: [], 2: [], 3: [], 4: [], 5: []}


Embedding broken by local gate.

In [12]:
# Custom placement
placement_dict10 = dict()
for i in range(6):
    placement_dict10[i] = i // 2
placement_dict10[6] = 0
placement_dict10[7] = 0
placement_dict10[8] = 0
placement_dict10[9] = 0
placement_dict[41] = 0
placement_dict[42] = 0
placement_dict[43] = 0
placement_dict[44] = 0
placement10 = Placement(placement_dict10)

# Build circuit
circ10 = Circuit(6)
circ10.add_gate(cz, [0,2])
circ10.H(0)
circ10.add_gate(cz, [0,2])
circ10.Rz(1, 0)
circ10.add_gate(cz, [0,1])
circ10.H(0)
circ10.add_gate(cz, [0,2])
circuit.add_circuit(circ10, mapping)
render_circuit_jupyter(circ10)

# Make hypergraph of circuit
h_circ10 = HypergraphCircuit(circ10)

# Pass into PacMan
pacman10 = PacMan(h_circ10, placement10)

print(pacman10.hopping_packets)

{0: [], 1: [], 2: [], 3: [], 4: [], 5: []}


In [26]:
placement = Placement(placement_dict)
render_circuit_jupyter(circuit)
print(len(circuit.get_commands()))
print(circuit.n_gates_of_type(OpType.CU1))
pacman = PacMan(HypergraphCircuit(circuit), placement)

80
39


In [21]:
print(pacman.get_all_packets())
print(pacman.merged_packets)


[P0, P1, P2, P3, P4, P5, P6, P7, P8, P9, P10, P11, P12, P13, P14, P15, P16, P17, P18, P19, P20, P21, P22, P23, P24, P25, P26, P27, P28, P29, P30, P31, P32, P33]
{0: [(P0, P1, P2, P4, P7, P10, P13, P16, P19, P22), (P3, P5), (P6, P8, P9, P11, P12, P14), (P15, P17), (P18, P20), (P21, P23), (P24,), (P25,), (P26,), (P27,), (P28,)], 1: [(P29,)], 2: [(P30,)], 3: [(P31,)], 4: [(P32,)], 5: [(P33,)]}


Graph methods are useful for analysing the ebit cost of these packings.  
We can use the method `.get_nx_graph_merged()` to return a graph representation of the circuit.

Note that this method also returns all the nodes on one half of the bipartite graph (conventionally called the top half). This is because the graph is likely highly disconnected, so `NetworkX` requires this to avoid ambiguous solutions to certain methods.